In [145]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.metrics import mean_squared_error

In [146]:
#Join iata data
iata = pd.read_csv('data/IATA_US_CODES.csv')
iata['IATA Code']=iata['IATA Code'].astype(str)
iata['IATA Code']=iata['IATA Code'].str.rstrip()
iata=iata.set_index('IATA Code')
iata_dep=iata.add_suffix('_Dep')
iata_arr=iata.add_suffix('_Arr')
iata['2018']=iata['2018'].str.replace(',','')
iata['2017']=iata['2017'].str.replace(',','')
iata['2016']=iata['2016'].str.replace(',','')
iata['2015']=iata['2015'].str.replace(',','')
iata['2018']=iata['2018'].str.replace('\xa0','')
iata['2017']=iata['2017'].str.replace('\xa0','')
iata['2016']=iata['2016'].str.replace('\xa0','')
iata['2015']=iata['2015'].str.replace('\xa0','')
iata['State']=iata['State'].str.replace('\xa0','')
    

In [147]:
iata

,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015
IATA Code,,,,,,,,
ATL,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
LAX,2,Los Angeles International Airport,Los Angeles,CA,42626783,41232432,39636042,36351226
ORD,3,O'Hare International Airport,Chicago,IL,39874879,38593028,37589899,36305668
DFW,4,Dallas/Fort Worth International Airport,Dallas/Fort Worth,TX,32800721,31816933,31283579,31589832
DEN,5,Denver International Airport,Denver,CO,31363573,29809097,28267394,26280043
JFK,6,John F. Kennedy International Airport,New York,NY,30541459,29533154,29239151,27782369
SFO,7,San Francisco International Airport,San Francisco,CA,27794154,26900048,25707101,24190549
SEA,8,Seattle–Tacoma International Airport,Seattle/Tacoma,WA,24894338,22639124,21887110,21231781
LAS,9,McCarran International Airport,Las Vegas,NV,23655285,23364393,22833267,21824231


In [148]:
iata['Major city served']=iata['Major city served'].str.replace('/Newark','')
iata['Major city served']=iata['Major city served'].str.replace('/St. Paul','')
iata['Major city served']=iata['Major city served'].str.replace('Baltimore/','')
iata['Major city served']=iata['Major city served'].str.replace('/Tacoma','')
iata['Major city served']=iata['Major city served'].str.replace('/Forth Worth','')
iata['Major city served']=iata['Major city served'].str.replace(', D.C.','')
iata

,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015
IATA Code,,,,,,,,
ATL,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
LAX,2,Los Angeles International Airport,Los Angeles,CA,42626783,41232432,39636042,36351226
ORD,3,O'Hare International Airport,Chicago,IL,39874879,38593028,37589899,36305668
DFW,4,Dallas/Fort Worth International Airport,Dallas/Fort Worth,TX,32800721,31816933,31283579,31589832
DEN,5,Denver International Airport,Denver,CO,31363573,29809097,28267394,26280043
JFK,6,John F. Kennedy International Airport,New York,NY,30541459,29533154,29239151,27782369
SFO,7,San Francisco International Airport,San Francisco,CA,27794154,26900048,25707101,24190549
SEA,8,Seattle–Tacoma International Airport,Seattle,WA,24894338,22639124,21887110,21231781
LAS,9,McCarran International Airport,Las Vegas,NV,23655285,23364393,22833267,21824231


In [149]:
data_weather = pd.read_csv('submissions/starting_kit/external_data.csv')


In [150]:
data_weather=data_weather.join(iata, on='AirPort', how='left')
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593


In [151]:
us_cities = pd.read_csv('data/uscities.csv')
us_cities['state_id'].unique()
data_weather['State'].unique()

array(['GA', 'IL', 'CA', 'TX', 'CO', 'NY', 'NC', 'NV', 'AZ', 'FL', 'NJ',
       'WA', 'MN', 'MI', 'PA', 'MA'], dtype=object)

In [152]:
data_weather=data_weather.merge(us_cities, left_on=['Major city served', 'State'],right_on=['city', 'state_id'], how='left')

In [153]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121.0,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1.0,30334 30331 30332 30309 30308 30305 30307 3030...,1.840016e+09
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121.0,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1.0,30334 30331 30332 30309 30308 30305 30307 3030...,1.840016e+09
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121.0,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1.0,30334 30331 30332 30309 30308 30305 30307 3030...,1.840016e+09
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121.0,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1.0,30334 30331 30332 30309 30308 30305 30307 3030...,1.840016e+09
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121.0,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1.0,30334 30331 30332 30309 30308 30305 30307 3030...,1.840016e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061.0,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1.0,11229 11226 11225 11224 11222 11221 11220 1138...,1.840060e+09
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061.0,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1.0,11229 11226 11225 11224 11222 11221 11220 1138...,1.840060e+09
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061.0,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1.0,11229 11226 11225 11224 11222 11221 11220 1138...,1.840060e+09
11038,2013-03-04,LGA

In [154]:
data_weather.to_csv('external_data2.csv')